# CRM Query Engine Test
This notebook demonstrates the implementation and usage of the SQL query engine.

In [ ]:
from currensee.schema.schema import PostgresTables
from currensee.query_engines.sql_workflow.utils import create_sql_workflow
from currensee.utils.db_utils import create_pg_engine
from currensee.query_engines.workflow_descriptions import (
    crm_portfolio_table_desc,
    crm_client_alignment_table_desc,
    crm_client_info_table_desc,
    crm_employees_table_desc,
    crm_fund_details_desc,
)

In [ ]:
# required to run asynchronous code

import nest_asyncio

nest_asyncio.apply()

## Create the SQL Workflow

The SQL workflow can take the following parameters:

1. source_db: the name of the database where the table is stored (e.g. `crm`)
2. source_tables: a list of the name(s) of the table(s) that we want the query engine to have access to
  * note that multiple tables can be passed - this is if you want the query engine to try to join tables
    in the queries that may have relationships to one another
  * THIS IS LEVEL 2!! So do not attempt until you get the hang of just using one table at a time!!
    
3. table_descriptions: a list of the description(s) of the table(s) passed above
4. text_to_sql_tmpl: a string containing the prompt telling the LLM how to produce the SQL query from the text given
   * defaults to the variable `text_to_sql_tmpl` defined in `currensee.query_engines.prompting.py`
   * you may override this by passing in your own string
5. response_synthesis_prompt_str: a string containing the prompt telling the LLM how to synthesize the final response from the SQL table(s)
   * defaults to the variable `response_synthesis_prompt_str` defined in `currensee.query_engines.prompting.py`
   * you may override this by passing in your own string
6. model: the name of the model to use for all of the tasks
   * defaults to `gemini-1.5-flash`
   * you may override this with any of the models defined at https://ai.google.dev/gemini-api/docs/models#model-variations using the string with dashes defined in the "Model variant" column.
   * **BE VERY CAREFUL TO PAY ATTENTION TO THE PRICING!!!!!** I recommend that you use the default model until you understand the other models better!!!
7. temperature: the temperature parameter to pass to the model
   * default is 0.0
   * the higher the temperature, the more creative it is. Recommend keeping low for the SQL query generation.

### Below is the default defined in `prompting.py`

In [ ]:
text_to_sql_tmpl = """
    Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
    You can order the results by the find_date column (from earliest to latest) to return the most interesting examples in the database.

    GUIDELINES:
    * Never query for all the columns from a specific table, only ask for a few relevant columns given the question.
    * Pay attention to use only the column names that you can see in the schema description.
    * Be careful to not query for columns that do not exist.
    * Pay attention to which column is in which table.
    * Make sure to filter on all criteria mentioned in the query.
    * If using a LIMIT to restrict the results, make sure it comes only in the end of the query.

    IMPORTANT NOTE:
    * Use the ~* operator instead of = when filtering with WHERE on text columns.
    * Add word boundaries '\y' to the beginning and end of each search term in the query.

    You are required to use the following format, each taking one line:

    Question: Question here
    SQLQuery: SQL Query to run
    SQLResult: Result of the SQLQuery
    Answer: Final answer here

    Only use tables listed below.
    {schema}

    Question: {query_str}
    SQLQuery:

"""

### Below is the default defined in `prompting.py`

In [ ]:
response_synthesis_prompt_str = """

    Query: {query_str}
    SQL: {sql_query}
    SQL Response: {context_str}

    IMPORTANT INSTRUCTIONS:
    * If SQL Response is empty or 0, apologise and mention that you could not find
     examples to answer the query.
    * In such cases, kindly nudge the user towards providing more details or refining
    their search.
    * Additionally, you can tell them to rephrase specific keywords.
    * Do not explicitly state phrases such as 'based on the SQL query executed' or related
     references to context in your Response.
    * Never mention the underlying sql query, or the underling sql tables and other database elements
    * Never mention that sql was used to answer this question

    Considering the IMPORTANT INSTRUCTIONS above, create an response using the information
    returned from the database and no prior knowledge.


    Response:
"""

### Define the DB information
**IMPORTANT**: The table names MUST be lowercase in order for the engine to find them.

In [ ]:
source_db = "crm"
table_description_mapping = {
    "employees": crm_employees_table_desc,
    "portfolio": crm_portfolio_table_desc,
    "fund_detail": crm_fund_details_desc,
    "client_alignment": crm_client_alignment_table_desc,
    "clients_contact": crm_client_info_table_desc,
}

In [ ]:
sql_workflow = create_sql_workflow(
    source_db=source_db,
    table_description_mapping=table_description_mapping,
    text_to_sql_tmpl=text_to_sql_tmpl,
    response_synthesis_prompt_str=response_synthesis_prompt_str,
)

# Test Queries

In [ ]:
query = "Who works for bankwell?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "How many people work for bankwell in the Sales department?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "what financial instruments does Mariott own?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "how many funds does Broadcom own? What types of funds are they?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "What is Broadcom's total balance? What fund does broadcom have the most money in, and how much is there?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "How much money does broadcom have in each fund?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "what employees work on the mariott client account?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "what is the client email for Mariott?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "what is the stocks does the client Broadcom have in their funds?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "what is the stocks does Mariott have in their equity funds?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "What stocks are in the FTIEX equity fund?"
result = await sql_workflow.run(query=query)
result

In [ ]:
query = "What funds does Walmart own? How much money is in each fund?"
result = await sql_workflow.run(query=query)
result

# Check Answers accuracy

In [ ]:
from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

In [ ]:
DB_NAME = "crm"
engine = create_pg_engine(db_name=DB_NAME)

In [ ]:
pd.read_sql("SELECT count(*) FROM Employees", con=engine)

In [ ]:
pd.read_sql("SELECT * FROM portfolio where Company = 'Mariott' ", con=engine)

In [ ]:
pd.read_sql("SELECT * FROM portfolio where Company = 'Broadcom' ", con=engine)

In [ ]:
pd.read_sql("SELECT * FROM client_alignment where Company = 'Mariott' ", con=engine)

In [ ]:
pd.read_sql("SELECT * FROM clients_contact where Company = 'Mariott' ", con=engine)

In [ ]:
pd.read_sql(
    """SELECT T1.position_name 
FROM fund_detail AS T1 
JOIN portfolio AS T2 ON 
T1.fund  =  T2.symbol 
WHERE T2.company = 'Mariott' AND T2.fund_type = 'Equity Fund'
""",
    con=engine,
)

"""I found the following stocks held in Marriott's equity funds:  NVDIA Corp, Apple Inc, Amazon.com Inc, 
Microsoft Corp, AT&T Inc, Bank of America Corp, Alphabet Inc Class A, Walmart Inc, Exxon Mobil Corp, and Intel Corp"""